In [1]:
# Import libraries and api keys
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [2]:
# Load CSV file created in WeatherPy
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port lincoln,-34.7333,135.8667,15.37,67,100,3.67,AU,1698416478
1,1,wailua homesteads,22.0669,-159.3780,21.24,78,0,3.09,US,1698416478
2,2,dudinka,69.4058,86.1778,-12.96,99,69,1.34,RU,1698416478
3,3,adamstown,-25.0660,-130.1015,22.16,81,100,2.40,PN,1698416479
4,4,utrik,11.2278,169.8474,27.89,78,64,3.86,MH,1698416479


In [5]:
pip install geoviews

  Obtaining dependency information for geoviews from https://files.pythonhosted.org/packages/7c/b1/c27b294bc3bfc06d7721082568f53d076918a039864afc656d6d237fe08b/geoviews-1.10.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/509.8 kB ? eta -:--:--
   --------------------------------------  501.8/509.8 kB 15.4 MB/s eta 0:00:01
   --------------------------------------- 509.8/509.8 kB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Plot cities on the holoview plot

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [7]:
# Narrow down cities that fit our vacation criteria 

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,wailua homesteads,22.0669,-159.3780,21.24,78,0,3.09,US,1698416478
33,33,yigo village,13.5361,144.8886,26.62,93,0,3.09,GU,1698416483
38,38,haiku-pauwela,20.9219,-156.3051,22.80,83,0,2.57,US,1698416484
52,52,saint-pierre,-21.3393,55.4781,25.82,78,0,2.06,RE,1698416486
85,85,charleston,32.7766,-79.9309,25.57,84,0,2.06,US,1698416285
170,170,cannonvale,-20.2833,148.7000,21.40,70,0,4.49,AU,1698416502
195,195,pula,44.8683,13.8481,21.31,68,0,4.12,HR,1698416396
199,199,fort bragg,35.1390,-79.0060,23.23,85,0,0.89,US,1698416506
230,230,holualoa,19.6228,-155.9522,24.37,73,0,3.58,US,1698416511
255,255,lihue,21.9789,-159.3672,21.58,78,0,3.09,US,1698416285


In [8]:
# Figure out hotel arrangements

hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,wailua homesteads,US,22.0669,-159.3780,78,
33,yigo village,GU,13.5361,144.8886,93,
38,haiku-pauwela,US,20.9219,-156.3051,83,
52,saint-pierre,RE,-21.3393,55.4781,78,
85,charleston,US,32.7766,-79.9309,84,
170,cannonvale,AU,-20.2833,148.7000,70,
195,pula,HR,44.8683,13.8481,68,
199,fort bragg,US,35.1390,-79.0060,85,
230,holualoa,US,19.6228,-155.9522,73,
255,lihue,US,21.9789,-159.3672,78,


In [9]:
# Find the first hotel located within 10,000 metres 

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

print("Starting hotel search")

# Iterate through hotel_df
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Build parameters
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make API request
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    # JSONify
    name_address = name_address.json()
    
    # Grab the first hotel from results
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
# Display data
hotel_df

Starting hotel search
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


,City,Country,Lat,Lng,Humidity,Hotel Name
1,wailua homesteads,US,22.0669,-159.3780,78,Hilton Garden Inn Kauai Wailua Bay
33,yigo village,GU,13.5361,144.8886,93,No hotel found
38,haiku-pauwela,US,20.9219,-156.3051,83,Inn At Mama's Fish House
52,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
85,charleston,US,32.7766,-79.9309,84,"Mills House Charleston, Curio Collection by Hi..."
170,cannonvale,AU,-20.2833,148.7000,70,Whitsundays Rainforest Retreat
195,pula,HR,44.8683,13.8481,68,Omir
199,fort bragg,US,35.1390,-79.0060,85,Airborne Inn Lodging
230,holualoa,US,19.6228,-155.9522,73,Kona Hotel
255,lihue,US,21.9789,-159.3672,78,Kauai Palms


In [10]:
# Plot holoview and show hotels on the map

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)